Automatic classification of stigmatizing mental illness articles in online news journals - April 2022 Author: Alina Yanchuk - alinayanchuk@ua.pt

### Table of contents:

* [1. Preprocessing](#chapter1)
    * [1.1 Requirements](#section_1_1)
    * [1.2 Imports](#section_1_2)
    * [1.3 Get data](#section_1_3)
    * [1.4 Process data](#section_1_4)

# 1. Preprocessing <a class="anchor" id="chapter1"></a>

Data cleaning/preparation for the next tasks - Classification, Topic Modeling and Exploratory Data Analysis.

## 1.1 Requirements <a class="anchor" id="section_1_1"></a>

In [4]:
pip install pandas

     |████████████████████████████████| 11.7 MB 355 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade nltk

     |████████████████████████████████| 1.5 MB 3.0 MB/s 
Note: you may need to restart the kernel to use updated packages.


## 1.2 Imports <a class="anchor" id="section_1_2"></a>

In [9]:
import pandas as pd

import re
import string

import nltk
nltk.download('stopwords')

import os

[nltk_data] Downloading package stopwords to /home/alina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1.3 Get data <a class="anchor" id="section_1_3"></a>

In [10]:
# Table only with news 'headline' and 'content'

pd.set_option('max_colwidth',150)
data = pd.read_csv ('data_labeled.csv')
data_text_eda = data # data prepared for Exploratory Analysis Tasks

data_text = data.loc[:, ['label', 'headline', 'content']] # data prepared for Classification tasks

In [11]:
# Search for null values

for col in data_text.columns:
    print(col, data_text[col].isnull().sum())

label 0
headline 0
content 0


In [12]:
# Remove rows with null content

data_text_eda = data_text_eda[data_text_eda['content'].notna()]
data_text = data_text[data_text['content'].notna()]

print("Total number of news: " + str(len(data_text)))

Total number of news: 978


In [13]:
# Join 'headline' and 'content'

for index, row in data_text_eda.iterrows():
    if isinstance(row['headline'], float): pass
    else: 
        if row['headline'] not in row['content']: row['content'] = row['headline'] + ' ' + row['content']
data_text_eda.drop('headline', axis = 1, inplace = True) # For EDA

for index, row in data_text.iterrows():
    if isinstance(row['headline'], float): pass
    else: 
        if row['headline'] not in row['content']: row['content'] = row['headline'] + ' ' + row['content']
data_text.drop('headline', axis = 1, inplace = True) # For Classification and TM
data_text.head()

,label,content
0,literal,"Prisão perpétua para homem que tentou assassinar senadora No dia 8 de Janeiro de 2011 Jared Loughner tentou matar, sem sucesso, a então senadora G..."
1,literal,"John Nash, matemático de Uma Mente Brilhante, morre em acidente John Nash, o matemático e Nobel da Economia que foi retratado por Russell Crowe no..."
2,estigma,O mito da reeleição mínima garantida Cavaco sairá desta campanha pior do que entrou. Os casos das acções na SLN ou da casa de férias no Algarve nã...
3,literal,"Morreu Rita Levi-Montalcini, a grande dama da ciência italiana A cientista e senadora italiana Rita Levi-Montalcini, Prémio Nobel da Medicina de 1..."
4,literal,"Por trás da porta amarela, há um homem com problemas psicológicos Pub Ninguém sabe o que ele está a fazer. Ninguém sabe em que é que pensa, o que..."


## 1.4 Process data <a class="anchor" id="section_1_4"></a>

In [14]:
# Cleaning

irrelevant = ["pub", "pág", "reportagem", "clique", "subscrever", "partilhar", "dn", "tvi", "sic", "rtp", "cartaz", "tv"] # list with irrelevant words/expressions do the news content

stopwords = nltk.corpus.stopwords.words('portuguese') # pt stopwords

sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle') # pt sentence tokenizer

#nlp = stanza.Pipeline('pt') # pt stanza

def clean_text(text):

    text = text.lower() # lowercase
    text = re.sub(r'http\S+', '', text) # URLs removal
    text = re.sub('\[.*?\]', '', text) # remove text in square brackets
    text = re.sub(r"\([^()]*\)", '', text) # remove text within parentheses
    text = re.sub(r'-(me|te|se|nos|vos|o|os|a|as|lo|los|la|las|lhe|lhes|lha|lhas|lho|lhos|no|na|nas|mo|ma|mos|mas|to|ta|tos|tas)', '', text) # remove pronouns at the end of the words
    text = ' '.join([sentence for sentence in sent_tokenizer.tokenize(text) if len(sentence) > 15]) 
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
    text = re.sub('[‘’“”«|»…]', '', text)
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = ' '.join([word for word in text.split() if word not in stopwords]) # remove stopwords
    text = ' '.join([word for word in text.split() if len(word) > 2]) # remove words with length <= 2
    text = re.sub(r'Pág.*por pág$', '', text) # remove irrelevant text
    text = ' '.join([word for word in text.split() if word not in irrelevant]) 
    #text = nlp(text)
    #text = ' '.join([word.lemma for sent in text.sentences for word in sent.words]) 

    return text

data_text_eda["content"] = data_text_eda.content.apply(lambda x: clean_text(x))
data_clean_eda = data_text_eda

data_text["content"] = data_text.content.apply(lambda x: clean_text(x))
data_clean = data_text
data_clean.head()

,label,content
0,literal,prisão perpétua homem tentou assassinar senadora dia janeiro jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido ...
1,literal,john nash matemático mente brilhante morre acidente john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu sábad...
2,estigma,mito reeleição mínima garantida cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudess...
3,literal,morreu rita levintalcini grande dama ciência italiana cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste domingo ano...
4,literal,trás porta amarela homem problemas psicológicos ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filh...


In [15]:
# Count the number of vales in each of the two labels/categories

data_clean['label'].value_counts()

estigma    509
literal    469
Name: label, dtype: int64

In [16]:
# Associate label names with binary numerical values

data_clean['label'] = data_clean['label'].factorize()[0]
data_clean.head()

,label,content
0,0,prisão perpétua homem tentou assassinar senadora dia janeiro jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido ...
1,0,john nash matemático mente brilhante morre acidente john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu sábad...
2,1,mito reeleição mínima garantida cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudess...
3,0,morreu rita levintalcini grande dama ciência italiana cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste domingo ano...
4,0,trás porta amarela homem problemas psicológicos ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filh...


In [17]:
# Dictionary that stores label/index and index/label conversion

index_to_label = {0: 'literal', 1: 'estigma'}
label_to_index = {'literal': 0, 'estigma': 1}
label_to_index

{'literal': 0, 'estigma': 1}

In [18]:
# Add a column with the journal title to the dataset for EDA

data_clean_eda.insert(3, 'journalTitle', "")

def add_journal_title(journal):

    journal_title = " "
     
    if journal in ["publico.pt", "www.publico.pt", "jornal.publico.pt", "dossiers.publico.pt", "desporto.publico.pt", "www.publico.clix.pt", "digital.publico.pt", "blogues.publico.pt", "economia.publico.pt", "m.publico.pt", "ultimahora.publico.pt"]:
        journal_title = "Público"
    elif journal=="observador.pt": 
        journal_title = "Observador"
    elif journal in ["www.dn.pt", "dn.sapo.pt", "www.dn.sapo.pt"]:
        journal_title = "Diário de Notícias"
    elif journal in ["expresso.pt", "aeiou.expresso.pt", "expresso.sapo.pt"]:
        journal_title = "Expresso"
    elif journal in ["www.correiomanha.pt", "www.correiodamanha.pt", "www.cmjornal.xl.pt", "www.cmjornal.pt"]:
        journal_title = "Correio da Manhã"
    elif journal in ["www.jn.pt", "jn.pt", "jn.sapo.pt"]:
        journal_title = "Jornal de Notícias"
    elif journal in ["abola.pt", "www.abola.pt", "abola.pt:80"]:
        journal_title = "A Bola"
    elif journal in ["www.sabado.pt", "www.sabado.pt:80", "www.sabado.xl.pt", "www.sabado.xl.pt:80", "sabado.pt"]:
        journal_title = "Sábado"
    elif journal in ["visaoonline.clix.pt:80", "visao.clix.pt:80", "aeiou.visao.pt", "visao.sapo.pt"]:
        journal_title = "Visão"

    return journal_title
    
data_clean_eda["journalTitle"] = data_clean_eda.journal.apply(lambda x: add_journal_title(x))
data_clean_eda.head()

,ID,label,journal,journalTitle,content,authors,publishDate,archiveDate,linkToArchive
0,0,literal,publico.pt,Público,dia janeiro jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido democrata arizona matou seis pessoas quase dois a...,[],None,2012-12-30,https://arquivo.pt/wayback/20121230181331/http://publico.pt/mundo/noticia/homem-que-tentou-assassinar-senadora-norteamericana-condenado-a-sete-pri...
1,1,literal,publico.pt,Público,john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu sábado acidente autoestrada nova jérsia nash anos mulher ...,[],None,2016-01-17,https://arquivo.pt/wayback/20160117223452/http://publico.pt/economia/noticia/john-nash-matematico-de-uma-mente-brilhante-morre-em-acidente-1696715
2,2,estigma,publico.pt,Público,cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudesse ser apontada ficou ruído estra...,"['Nuno Ferreira Santos', 'Arquivo']",None,2011-01-21,https://arquivo.pt/wayback/20110121142608/http://publico.pt/Presidenciais2011/Noticias/Analise/o-mito-da-reeleicao-minima-garantida_1475944
3,3,literal,publico.pt,Público,cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste domingo anos casa roma corpo faz quer corpo mente disse entrevist...,['Clara Barata'],None,2013-01-17,https://arquivo.pt/wayback/20130117170513/http://publico.pt/ciencia/noticia/morreu-rita-levimontalcini-a-grande-dama-da-ciencia-italiana-1579018
4,4,literal,publico.pt,Público,ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filhos deixou ali sozinho nunca falou ninguém alguns...,[],None,2015-04-20,https://arquivo.pt/wayback/20150420143056/http://publico.pt/sociedade/noticia/por-tras-da-porta-amarela-ha-um-homem-com-problemas-psicologicos-166...


In [19]:
# Add a column with the journal title in the datatable for EDA

data_clean_eda.insert(8, 'year', "")


def add_year(year):

    new_year = " "
     
    if "1996-" in year: new_year="1996"
    elif "1997-" in year: new_year="1997"
    elif "1998-" in year: new_year="1998"
    elif "1999-" in year: new_year="1999"
    elif "2000-" in year: new_year="2000"
    elif "2001-" in year: new_year="2001"
    elif "2002-" in year: new_year="2002"
    elif "2003-" in year: new_year="2003"
    elif "2004-" in year: new_year="2004"
    elif "2005-" in year: new_year="2005"
    elif "2006-" in year: new_year="2006"
    elif "2007-" in year: new_year="2007"
    elif "2008-" in year: new_year="2008"
    elif "2009-" in year: new_year="2009"
    elif "2010-" in year: new_year="2010"
    elif "2011-" in year: new_year="2011"
    elif "2012-" in year: new_year="2012"
    elif "2013-" in year: new_year="2013"
    elif "2014-" in year: new_year="2014"
    elif "2015-" in year: new_year="2015"
    elif "2016-" in year: new_year="2016"
    elif "2017-" in year: new_year="2017"
    elif "2018-" in year: new_year="2018"
    elif "2019-" in year: new_year="2019"
    elif "2020-" in year: new_year="2020"
    elif "2021-" in year: new_year="2021"

    return new_year

data_clean_eda["year"] = data_clean_eda.archiveDate.apply(lambda x: add_year(x))
data_clean_eda.head()

,ID,label,journal,journalTitle,content,authors,publishDate,archiveDate,year,linkToArchive
0,0,literal,publico.pt,Público,dia janeiro jared loughner tentou matar sucesso então senadora gabrielle giffords eleita partido democrata arizona matou seis pessoas quase dois a...,[],None,2012-12-30,2012,https://arquivo.pt/wayback/20121230181331/http://publico.pt/mundo/noticia/homem-que-tentou-assassinar-senadora-norteamericana-condenado-a-sete-pri...
1,1,literal,publico.pt,Público,john nash matemático nobel economia retratado russell crowe filme mente brilhante morreu sábado acidente autoestrada nova jérsia nash anos mulher ...,[],None,2016-01-17,2016,https://arquivo.pt/wayback/20160117223452/http://publico.pt/economia/noticia/john-nash-matematico-de-uma-mente-brilhante-morre-em-acidente-1696715
2,2,estigma,publico.pt,Público,cavaco sairá desta campanha pior entrou casos acções sln casa férias algarve revelaram qualquer ilegalidade pudesse ser apontada ficou ruído estra...,"['Nuno Ferreira Santos', 'Arquivo']",None,2011-01-21,2011,https://arquivo.pt/wayback/20110121142608/http://publico.pt/Presidenciais2011/Noticias/Analise/o-mito-da-reeleicao-minima-garantida_1475944
3,3,literal,publico.pt,Público,cientista senadora italiana rita levintalcini prémio nobel medicina morreu neste domingo anos casa roma corpo faz quer corpo mente disse entrevist...,['Clara Barata'],None,2013-01-17,2013,https://arquivo.pt/wayback/20130117170513/http://publico.pt/ciencia/noticia/morreu-rita-levimontalcini-a-grande-dama-da-ciencia-italiana-1579018
4,4,literal,publico.pt,Público,ninguém sabe fazer ninguém sabe pensa come sobrevive dois anos desde mulher pânico levou dois filhos deixou ali sozinho nunca falou ninguém alguns...,[],None,2015-04-20,2015,https://arquivo.pt/wayback/20150420143056/http://publico.pt/sociedade/noticia/por-tras-da-porta-amarela-ha-um-homem-com-problemas-psicologicos-166...


In [20]:
# Store the cleaned and preprocessed data

parent = os.path.dirname(os.getcwd())

data_clean.to_pickle(parent+"/2.classification/data_preprocessed.pkl")
data_clean.to_pickle(parent+"/3.topic modeling/data_preprocessed_tm.pkl")
data_clean_eda.to_pickle(parent+"/4.exploratory data analysis/data_preprocessed_eda.pkl")